In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, QuantileTransformer, PowerTransformer, RobustScaler, MinMaxScaler

from wrangle import wrangle_telco
import split_scale

In [2]:
df = wrangle_telco()
df.head(1)

,customer_id,monthly_charges,tenure,total_charges
0,0013-SMEOE,109.7,71,7904.25


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1695 entries, 0 to 1694
Data columns (total 4 columns):
customer_id        1695 non-null object
monthly_charges    1695 non-null float64
tenure             1695 non-null int64
total_charges      1695 non-null float64
dtypes: float64(2), int64(1), object(1)
memory usage: 53.1+ KB


# Validate split_my_data()

In [4]:
train, test = split_scale.split_my_data(df)

In [5]:
train.head(1)

,customer_id,monthly_charges,tenure,total_charges
1469,8661-BOYNW,84.4,72,6096.45


In [6]:
train.shape

(1186, 4)

In [7]:
test.head(1)

,customer_id,monthly_charges,tenure,total_charges
252,1494-EJZDW,20.15,10,220.8


In [8]:
test.shape

(509, 4)

# Define my X, independent variables, and y, dependent/target variable

In [9]:
X_train = train[['tenure']]
X_test = test[['tenure']]
y_train = [['total_charges']]
y_test = [['total_charges']]

In [10]:
X_train.head(1)

,tenure
1469,72


In [11]:
X_test.head(1)

,tenure
252,10


In [12]:
print(X_train.shape)
print(X_test.shape)

(1186, 1)
(509, 1)


# Validate standard_scaler() function

In [13]:
scaler, X_train_scaled, X_test_scaled = split_scale.standard_scaler(X_train, X_test)


In [14]:
scaler

StandardScaler(copy=True, with_mean=True, with_std=True)

In [15]:
X_train_scaled.head()

,tenure
1469,0.838946
163,0.441745
392,0.044543
1546,0.838946
797,0.214772


In [16]:
X_test_scaled.head()

,tenure
252,-2.679126
632,0.328258
472,0.725460
1029,0.328258
910,0.782203


# Validate my scale_inverse() function

In [17]:
X_train_unscaled, X_test_unscaled = split_scale.scale_inverse(scaler, X_train_scaled, X_test_scaled)

In [18]:
X_train_unscaled.head()

,tenure
1469,72.0
163,65.0
392,58.0
1546,72.0
797,61.0


In [19]:
X_test_unscaled.head()

,tenure
252,10.0
632,63.0
472,70.0
1029,63.0
910,71.0


# Validate my uniform_scaler() function

In [ ]:
df = wrangle_telco()

In [ ]:
scaler, X_train_scaled, X_test_scaled = split_scale.uniform_scaler(X_train, X_test)

In [ ]:
scaler

In [ ]:
X_train_scaled.head()

In [ ]:
X_test_scaled.head()

### I can peek at the distribution of the train_scaled and test_scaled if I want.

In [ ]:
X_train_scaled.describe()

In [ ]:
X_test_scaled.describe()

# Validate my gaussian_scaler() function

In [ ]:
X_train.head(1)

In [ ]:
X_test.head(1)

In [ ]:
scaler, X_train_scaled, X_test_scaled = split_scale.gaussian_scaler(X_train, X_test)

In [ ]:
scaler

In [ ]:
X_train_scaled.head()

In [ ]:
X_test_scaled.head()

# Validate my min_max_scaler() function

In [ ]:
scaler, X_train_scaled, X_test_scaled = split_scale.min_max_scaler(X_train, X_test)

In [ ]:
scaler

In [ ]:
X_train_scaled.head()

In [ ]:
X_test_scaled.head()

# Validate my iqr_robust_scaler() function

In [ ]:
scaler, X_train_scaled, X_test_scaled = split_scale.iqr_robust_scaler(X_train, X_test)

In [ ]:
scaler

In [ ]:
X_train_scaled.head()

In [ ]:
X_test_scaled.head()

# Looks Good!